In [1]:
import json
import csv
import tweepy
import re
import pandas as pd

In [2]:
consumer_key='JCzkEeG8SWEYOzSLYs58yOMhG'
consumer_secret='WovtzcbhjV5yHqdoIeFwChQwuIMlIMnAzV9wH66E3GJkin6fFQ'
access_token='1357864325743075330-ApNiBiT577Kd7ZmgOsV9qTdwRuhkeH'
access_token_secret='yLmwbrdCqYPVk5DI0aH7vdqkW2gLBcEc8f9IVCKsuNLqL'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

#initialize Tweepy API
api = tweepy.API(auth)

In [4]:
import csv

with open(r'D:\tweet_data.csv','w') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['timestamp', 'tweet_text', 'username', 'followers_count','retweet_count'])
    for tweet in tweepy.Cursor(api.search, q='COVID VACCINE -filter:hashtags', \
                                   lang="en", tweet_mode='extended').items(1000):
        writer.writerow([tweet.created_at, tweet.full_text.replace('\n',' ').encode('utf-8'), tweet.user.screen_name.encode('utf-8'),tweet.user.followers_count, tweet.retweet_count])

## Read Data

In [5]:
data = pd.read_csv(r'D:\tweet_data.csv')

In [6]:
data.head()

,timestamp,tweet_text,username,followers_count,retweet_count
0,2021-02-13 01:53:57,b'RT @CNN: Biden says the US will have enough ...,b'mercury_22',54,596
1,2021-02-13 01:53:49,b'RT @hildabast: Chile: Managed diversified pr...,b'ggrillo93',351,3
2,2021-02-13 01:53:46,b'RT @SeedSparkLive: Make Schools More Human h...,b'jrosspeters',1012,1
3,2021-02-13 01:53:43,b'RT @MattHancock: Fantastic that 89% of peopl...,b'DR7fu2pharoh',16,80
4,2021-02-13 01:53:42,b'RT @gmanews: People who get a COVID-19 vacci...,b'lesismore333',27,7


In [7]:
data.shape

(1000, 5)

## Create target column

As the retweet count is less, I will use followers_count to generate the target class.

In [8]:
def get_target_column(followers_count):
    
    if followers_count > 1000:
        return 'Class A'
    else:
        return 'Class B'
    
data['target'] = data['followers_count'].apply(get_target_column)

In [9]:
data

,timestamp,tweet_text,username,followers_count,retweet_count,target
0,2021-02-13 01:53:57,b'RT @CNN: Biden says the US will have enough ...,b'mercury_22',54,596,Class B
1,2021-02-13 01:53:49,b'RT @hildabast: Chile: Managed diversified pr...,b'ggrillo93',351,3,Class B
2,2021-02-13 01:53:46,b'RT @SeedSparkLive: Make Schools More Human h...,b'jrosspeters',1012,1,Class A
3,2021-02-13 01:53:43,b'RT @MattHancock: Fantastic that 89% of peopl...,b'DR7fu2pharoh',16,80,Class B
4,2021-02-13 01:53:42,b'RT @gmanews: People who get a COVID-19 vacci...,b'lesismore333',27,7,Class B
...,...,...,...,...,...,...
995,2021-02-13 01:26:16,"b'RT @CarlosGSmith: ""All of this confusion [wi...",b'joan_sheba',1767,15,Class A
996,2021-02-13 01:26:16,"b'RT @martinmorrey: @BorisJohnson ""My concern,...",b'BarbaraGoggin4',907,6,Class B
997,2021-02-13 01:26:15,"b'RT @ALEXNEWMAN_JOU: Despite my pleading, dad...",b'Freeplanet7',813,435,Class B
998,2021-02-13 01:26:14,b'RT @NPR: President Biden has finalized deals...,b'DBChirpy',872,366,Class B


## Preprocess the tweets

We will use tweet-preprocessor 0.6.0 package in python. Lets install it.

In [10]:
!pip install tweet-preprocessor

In [11]:
import preprocessor as p

data['cleaned_text'] = data['tweet_text'].apply(p.clean)

In [12]:
data[['tweet_text', 'cleaned_text']]

,tweet_text,cleaned_text
0,b'RT @CNN: Biden says the US will have enough ...,b'RT : Biden says the US will have enough Covi...
1,b'RT @hildabast: Chile: Managed diversified pr...,b'RT : Chile: Managed diversified pre-ordering...
2,b'RT @SeedSparkLive: Make Schools More Human h...,b'RT : Make Schools More Human '
3,b'RT @MattHancock: Fantastic that 89% of peopl...,b'RT : Fantastic that % of people say they wou...
4,b'RT @gmanews: People who get a COVID-19 vacci...,b'RT : People who get a COVID-19 vaccine can s...
...,...,...
995,"b'RT @CarlosGSmith: ""All of this confusion [wi...","b'RT : ""All of this confusion [with ] has real..."
996,"b'RT @martinmorrey: @BorisJohnson ""My concern,...","b'RT : ""My concern, as a virologist, is that i..."
997,"b'RT @ALEXNEWMAN_JOU: Despite my pleading, dad...","b'RT : Despite my pleading, dad took the covid..."
998,b'RT @NPR: President Biden has finalized deals...,b'RT : President Biden has finalized deals to ...


As you can observe, the preprocessor cleans all the unwanted urls, emojis etc. Now lets filter out the columns that we know would impact the training.

In [13]:
cleaned_data = data[['cleaned_text','target']]

In [14]:
cleaned_data.head()

,cleaned_text,target
0,b'RT : Biden says the US will have enough Covi...,Class B
1,b'RT : Chile: Managed diversified pre-ordering...,Class B
2,b'RT : Make Schools More Human ',Class A
3,b'RT : Fantastic that % of people say they wou...,Class B
4,b'RT : People who get a COVID-19 vaccine can s...,Class B


## Convert text to vectors

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = cleaned_data[['cleaned_text']]
y = cleaned_data[['target']]

In [16]:
X.shape

(1000, 1)

In [17]:
y.shape

(1000, 1)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
td = TfidfVectorizer(max_features = 4500)
X = td.fit_transform(X['cleaned_text'].to_list()).toarray()

## Spliting data into train and test sets

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X.shape

(1000, 3038)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=100)

## Train a machine learning model

In [29]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

C:\Users\satis\Anaconda3\envs\py369\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [30]:
y_pred = classifier.predict(X_test)

## Evaluate the Model

In [31]:
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(y_test, y_pred)

print('\n Accuracy: ', accuracy_score(y_test, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)


 Accuracy:  0.65

Classification Report

               precision    recall  f1-score   support

     Class A       0.76      0.14      0.23       116
     Class B       0.64      0.97      0.77       184

    accuracy                           0.65       300
   macro avg       0.70      0.56      0.50       300
weighted avg       0.69      0.65      0.56       300

